# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data_ZR/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vilyuchinsk,52.9306,158.4028,17.15,75,89,1.70,RU,1719655797
1,1,georgetown,5.4112,100.3354,31.96,70,20,3.09,MY,1719655791
2,2,winnemucca,40.9730,-117.7357,19.03,32,0,6.17,US,1719655803
3,3,linghai,41.1653,121.3667,25.89,64,100,4.79,CN,1719655805
4,4,ulsteinvik,62.3432,5.8487,15.42,81,94,8.05,NO,1719655807


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

#Apply filters to narrow down the DataFrame
filtered_city_data_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display the filtered DataFrame
filtered_city_data_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
111,111,maraba,-5.3686,-49.1178,25.10,94,0,0.00,BR,1719656098
150,150,colonia,50.9333,6.9500,25.23,49,0,1.03,DE,1719656006
190,190,laguna,38.4210,-121.4238,21.08,57,0,2.06,US,1719656292
216,216,homosassa springs,28.8036,-82.5759,24.52,82,0,0.00,US,1719656141
220,220,beroroha,-21.6667,45.1667,26.98,24,0,1.81,MG,1719656392
248,248,tsiombe,-25.3000,45.4833,23.97,37,0,3.19,MG,1719656466
251,251,natal,-5.7950,-35.2094,23.12,88,0,4.12,BR,1719656332
302,302,borba,-4.3878,-59.5939,22.58,93,0,0.66,BR,1719656606
318,318,karliova,39.2983,41.0142,22.07,31,0,1.61,TR,1719656645
328,328,galich,58.3797,42.3475,26.76,42,0,0.38,RU,1719656670


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Create a new DataFrame called hotel_df with selected columns
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

# Add an empty column "Hotel Name"
hotel_df['Hotel Name'] = ""

# Display the new DataFrame
hotel_df



,City,Country,Lat,Lng,Humidity,Hotel Name
111,maraba,BR,-5.3686,-49.1178,94,
150,colonia,DE,50.9333,6.9500,49,
190,laguna,US,38.4210,-121.4238,57,
216,homosassa springs,US,28.8036,-82.5759,82,
220,beroroha,MG,-21.6667,45.1667,24,
248,tsiombe,MG,-25.3000,45.4833,37,
251,natal,BR,-5.7950,-35.2094,88,
302,borba,BR,-4.3878,-59.5939,93,
318,karliova,TR,39.2983,41.0142,31,
328,galich,RU,58.3797,42.3475,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 
params = {
   'categories': 'accommodation.hotel',
    'limit': 1,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
maraba - nearest hotel: Portobelo
colonia - nearest hotel: Wasserturm Hotel Cologne
laguna - nearest hotel: Holiday Inn Express & Suites
homosassa springs - nearest hotel: Bella Oasis Hotel
beroroha - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
natal - nearest hotel: Natal Palace Hotel
borba - nearest hotel: No hotel found
karliova - nearest hotel: No hotel found
galich - nearest hotel: Русь-отель
hasaki - nearest hotel: Choshi Plaza Hotel
shelbyville - nearest hotel: No hotel found
namibe - nearest hotel: Nelsal Pensao
brandon - nearest hotel: Brandon Motor Lodge
ambovombe - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
olinda - nearest hotel: Fidji Motel
tercan - nearest hotel: Tercan Öğretmenevi


,City,Country,Lat,Lng,Humidity,Hotel Name
111,maraba,BR,-5.3686,-49.1178,94,Portobelo
150,colonia,DE,50.9333,6.9500,49,Wasserturm Hotel Cologne
190,laguna,US,38.4210,-121.4238,57,Holiday Inn Express & Suites
216,homosassa springs,US,28.8036,-82.5759,82,Bella Oasis Hotel
220,beroroha,MG,-21.6667,45.1667,24,No hotel found
248,tsiombe,MG,-25.3000,45.4833,37,No hotel found
251,natal,BR,-5.7950,-35.2094,88,Natal Palace Hotel
302,borba,BR,-4.3878,-59.5939,93,No hotel found
318,karliova,TR,39.2983,41.0142,31,No hotel found
328,galich,RU,58.3797,42.3475,42,Русь-отель


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Plotting the map with hover information
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1,
    color="City",
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)